In [1]:
import os, sys                               # system functions
import nipype.interfaces.io as nio           # Data i/o
from nipype.interfaces.io import DataSink
import nipype.interfaces.fsl as fsl          # fsl
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility
import nipype.algorithms.modelgen as model   # model generation
import errno
import glob
import pandas as pd
from collections import Counter
from patsy.contrasts import ContrastMatrix
import numpy as np

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')


230206-13:26:11,891 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.7.0


### Copes at level 2:
##### Model 0:
1. FS
2. SS
3. Go
4. FS-Go
5. FS-SS
6. SS-Go

##### Model 1:
1. response_left
2. response_right
3. response_left-response_right

In [2]:
# general set-up
project_folder = '/home/scotti/projects/3t_7t_sst_comparison'

dataset_ids = [x.split('/')[-1]for x in sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 'subject_level_model/*')))]
model_ns = ['0']
fwhms = ['base']
# fwhms = ['5p0']

if model_ns[0] == '0':
    contrasts = ['0','1','2','3','4','5']  # task from second level model
elif model_ns[0] == '1':
    contrasts = ['0','1','2'] # motor from second level model
if model_ns[0] == '2':
    contrasts = ['0','1','2','3','4','5']  # task from second level model

subject_ids = [x.split('/')[-1].split('-')[-1] for x in sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 
                                     'subject_level_model', '*', 'sub-*')))]

work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders', 'all_datasets')
slm_folder = os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 'subject_level_model')

template_brain = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
template_brain_mask = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

output_dir = os.path.join(project_folder, 'derivatives', "glm_feat_hp_sct", "group_level_model", 'all_datasets', f'fwhm-{fwhms[0]}',f'model-{model_ns[0]}')

templates = {'level2_cope': os.path.join(slm_folder, '*','sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-s*_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-cope.nii.gz'),
             'level2_varcope': os.path.join(slm_folder, '*','sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-s*_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-varcope.nii.gz'),
             'level2_tdof': os.path.join(slm_folder, '*','sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-s*_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-tdof_t.nii.gz')}


In [3]:
print(f"""
datasets : {dataset_ids}
model_n : {model_ns}
no. subs : {len(subject_ids)}
subs : {subject_ids}
work_dir : {work_dir}
slm folder : {slm_folder}
output dir : {output_dir}
contrasts : {contrasts}

## if uneven number of template / designs / subs the code will crash
no.copes = {len(glob.glob(templates['level2_cope'].format(dataset_id='*',subject_id='*',fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
no.varvopes = {len(glob.glob(templates['level2_varcope'].format(dataset_id='*',subject_id='*',fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
no.tdofs = {len(glob.glob(templates['level2_tdof'].format(dataset_id='*',subject_id='*',fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
""")


datasets : ['Leipzig_7T_GdH', 'Leipzig_7T_SM', 'NTNU_7T_SJSI', 'aron_3T', 'openfmri_3T']
model_n : ['0']
no. subs : 174
subs : ['BI3T', 'FMFT', 'GAIT', 'KCAT', 'KP6T', 'LV2T', 'MRCT', 'NM3T', 'PF5T', 'RSIT', 'SC1T', 'SPGT', 'TS6T', 'UM2T', 'VL1T', 'WW2T', 'ZK4T', '01', '02', '03', '05', '06', '07', '08', '09', '10', '11', '13', '15', '16', '17', '18', '002', '003', '005', '006', '007', '009', '011', '012', '014', '015', '016', '017', '018', '020', '021', '023', '024', '025', '026', '029', '031', '032', '033', '034', '035', '037', '038', '039', '040', '041', '042', '01', '02', '03', '04', '05', '06', '07', '09', '10', '11', '12', '13', '14', '15', '10159', '10171', '10206', '10217', '10228', '10235', '10249', '10273', '10274', '10280', '10290', '10292', '10304', '10316', '10321', '10325', '10329', '10339', '10340', '10345', '10347', '10356', '10361', '10365', '10388', '10429', '10438', '10440', '10448', '10455', '10460', '10471', '10478', '10487', '10492', '10506', '10517', '10523', '1

In [4]:
# ## make brain mask for flameo
# import nilearn
# from nilearn import plotting
# import nibabel as nib

# hdr = nib.load('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
# brain_data = hdr.get_fdata()
# brain_data[brain_data>0] = 1
# brain_mask = nib.Nifti1Image(brain_data, affine=hdr.affine, header=hdr.header)
# brain_mask.to_filename('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

In [5]:
##
workflow = pe.Workflow(name='feat_level3_sst_hp_alldatasets')
workflow.base_dir = work_dir
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# Identity
identity = pe.Node(util.IdentityInterface(fields=['contrast_n', 'model_n', 'fwhm']), name='identity')
identity.iterables = [('contrast_n', contrasts),
                      ('fwhm', fwhms),
                      ('model_n', model_ns)]

# Selector
selector = pe.Node(nio.SelectFiles(templates), name='selector')
workflow.connect(identity, 'contrast_n', selector, 'contrast_n')
workflow.connect(identity, 'fwhm', selector, 'fwhm')
workflow.connect(identity, 'model_n', selector, 'model_n')

## Merge copes, varcopes, masks
copemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                          name="copemerge")

varcopemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                       name="varcopemerge")

workflow.connect(selector, 'level2_cope', copemerge, 'in_files')
workflow.connect(selector, 'level2_varcope', varcopemerge, 'in_files')

In [6]:
# get dataset + subject ids
def get_subject_ids():
    import glob
    import os
    
    dataset_ids, subject_ids = zip(*[(x.split('/')[-2],x.split('/')[-1].split('-')[-1]) for x in sorted(glob.glob(os.path.join('/home/scotti/projects/3t_7t_sst_comparison', 
                             'derivatives', 'glm_feat_hp_sct', 'subject_level_model', '*', 'sub-*')))])
        
    return dataset_ids, subject_ids

# define node to output subjects
subject_id_getter = pe.Node(util.Function(output_names=['dataset_ids','subject_ids'],
                                          function=get_subject_ids),
                                          name='subject_id_getter')

# create design matrix witih dummy variables for the different datasets (experimental)
def get_design_matrix(second_level_contrast, dataset_ids, subject_ids): #=subject_ids):
    print(second_level_contrast)
    
    import numpy as np
    from patsy.contrasts import ContrastMatrix
    from collections import Counter
    
#     # code to generate correct simple dummy variables
#     def _name_levels(prefix, levels):
#         return ["[%s%s]" % (prefix, level) for level in levels]

#     class Simple(object):
#         def _simple_contrast(self, levels):
#             nlevels = len(levels)
#             contr = -1.0 / nlevels * np.ones((nlevels, nlevels - 1))
#             contr[1:][np.diag_indices(nlevels - 1)] = (nlevels - 1.0) / nlevels
#             return contr

#         def code_with_intercept(self, levels):
#             contrast = np.column_stack(
#                 (np.ones(len(levels)), self._simple_contrast(levels))
#             )
#             return ContrastMatrix(contrast, _name_levels("Simp.", levels))

#         def code_without_intercept(self, levels):
#             contrast = self._simple_contrast(levels)
#             return ContrastMatrix(contrast, _name_levels("Simp.", levels[:-1]))
        
    levels = [1,2,3,4,5]
#     contrast = Simple().code_without_intercept(levels)
    
#     dummies = [[-1,-1,-1,-1],[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]
    dummies = [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]
    
    # for regressors do intercept and dummy variables
    regressors = {#'intercept': [1 for x in range(len(subject_ids))],
                  'GdH': list(np.repeat([item[0] for item in dummies],list(Counter(dataset_ids).values()))),
                  'SM': list(np.repeat([item[1] for item in dummies],list(Counter(dataset_ids).values()))),
                  'SJSI': list(np.repeat([item[2] for item in dummies],list(Counter(dataset_ids).values()))),
                  'aron': list(np.repeat([item[3] for item in dummies],list(Counter(dataset_ids).values()))),
                  'openfmri': list(np.repeat([item[4] for item in dummies],list(Counter(dataset_ids).values())))}
    
    # contrasts (3rd-level), original way, this way calculates difference of each group and mean.
#     third_level_contrasts = [('Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [1.0, 0, 0, 0, 0]),
#                              ('-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 0, 0, 0, 0]),
#                              ('SM-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 1.0, 0, 0, 0]),
#                              ('SJSI-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 0, 1.0, 0, 0]),
#                              ('aron-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 0, 0, 1.0, 0]),
#                              ('openfmri-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 0, 0, 0, 1.0])
#                             ]
    
    # new method to calculate exact means so do not have to reconstruct later
    third_level_contrasts = [('Group mean', 'T', ['GdH', 'SM', 'SJSI', 'aron', 'openfmri'], [1.0, 1.0, 1.0, 1.0, 1.0]),
                             ('-Group mean', 'T', ['GdH', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, -1.0, -1.0, -1.0, -1.0]),
                             ('GdH mean', 'T', ['GdH', 'SM', 'SJSI', 'aron', 'openfmri'], [1.0, 0.0, 0.0, 0.0, 0.0]),
                             ('SM mean', 'T', ['GdH', 'SM', 'SJSI', 'aron', 'openfmri'], [0.0, 1.0, 0, 0, 0]),
                             ('SJSI mean', 'T', ['GdH', 'SM', 'SJSI', 'aron', 'openfmri'], [0.0, 0, 1.0, 0, 0]),
                             ('aron mean', 'T', ['GdH', 'SM', 'SJSI', 'aron', 'openfmri'], [0.0, 0, 0, 1.0, 0]),
                             ('openfmri mean', 'T', ['GdH', 'SM', 'SJSI', 'aron', 'openfmri'], [0.0, 0, 0, 0, 1.0])
                            ]

# Unsure which third level contrasts to use ?? fix
#     third_level_contrasts = [('Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [1.0, 0, 0, 0, 0]),
#                              ('-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 0, 0, 0, 0]),
#                              ('SM mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 1.0, 0, 0, 0]),
#                              ('-SM mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, -1.0, 0, 0, 0]),
#                              ('SJSI mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 1.0, 0, 0]),
#                              ('-SJSI mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, -1.0, 0, 0]),
#                              ('aron mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 0, 1.0, 0]),
#                              ('-aron mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 0, -1.0, 0]),
#                              ('openfmri mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 0, 0, 1.0]),
#                              ('-openfmri mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 0, 0, -1.0]),
#                              ]
    
    # fix this so can use different vairrance group: issue is that design matrix is not separable
    groups = list(np.repeat(levels, list(Counter(dataset_ids).values()))) # set different dataset as groups to estimate variance separately
#     groups = [1 for x in range(len(subject_ids))]
    
    return third_level_contrasts, regressors, groups

contrastgen_l3 = pe.Node(util.Function(input_names=['second_level_contrast', 'dataset_ids', 'subject_ids'],
                                       output_names=['third_level_contrasts', 'regressors', 'groups'],
                                       function=get_design_matrix),
                      name='contrastgen_l3')

level3model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l3model')

workflow.connect(subject_id_getter, 'dataset_ids', contrastgen_l3, 'dataset_ids')
workflow.connect(subject_id_getter, 'subject_ids', contrastgen_l3, 'subject_ids')
workflow.connect(identity, 'contrast_n', contrastgen_l3, 'second_level_contrast')
workflow.connect(contrastgen_l3, 'third_level_contrasts', level3model, 'contrasts')
workflow.connect(contrastgen_l3, 'regressors', level3model, 'regressors')
workflow.connect(contrastgen_l3, 'groups', level3model, 'groups')


In [7]:
# run flame 1 + 2
flameo = pe.Node(
    interface=fsl.FLAMEO(),
    name="flameo")

flameo.iterables = ('run_mode', ['flame1', 'flame12'])
flameo.inputs.mask_file = template_brain_mask
flameo.inputs.infer_outliers = False   # run with automatic outlier detection

workflow.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level3model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

In [8]:
## cluster thresholding
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')
smoothestimate.inputs.mask_file = template_brain_mask

workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

# get volume
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
get_volume.inputs.in_file = template_brain_mask

# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])]
grf_cluster.inputs.out_localmax_txt_file = True
grf_cluster.inputs.pthreshold = 0.05
grf_cluster.inputs.out_index_file = True
grf_cluster.inputs.out_threshold_file = True

def volume_convert(input):
    return int(input[0])

workflow.connect(get_volume, ('out_stat', volume_convert), grf_cluster, 'volume')
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

In [9]:
## Datasink, how to output the data
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=output_dir

# rename some files so they're not split over a bunch of folders
# depending on the FWHM used, the regexp might have to be adpated due to number of elements
if len(fwhms[0]) == 3:
    substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                             'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                            (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                              'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                            # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                            #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                           ]

elif len(fwhms[0]) == 4:
    substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{4})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                             'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                            (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{4})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                              'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                            # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                            #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                           ]   
    
elif len(fwhms[0]) == 5:
    substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{5})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                             'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                            (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{5})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                              'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                            # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                            #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                           ]    

datasink.inputs.regexp_substitutions = substitutions_regexp

## todo: substitutions
workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

## cluster results
workflow.connect(grf_cluster, 'threshold_file', datasink, 'third_level_model.grf_thresholded_zstats_file')
workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'third_level_model.grf_localmax_txt_file')
workflow.connect(grf_cluster, 'index_file', datasink, 'third_level_model.grf_cluster_indices')

In [10]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 24, 'memory_gb': 240})

230206-13:26:46,674 nipype.workflow INFO:
	 Workflow feat_level3_sst_hp_alldatasets_5p0 settings: ['check', 'execution', 'logging', 'monitoring']
230206-13:26:46,832 nipype.workflow INFO:
	 Running in parallel.
230206-13:26:46,889 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 240.00/240.00, Free processors: 24/24.
230206-13:26:47,12 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp_alldatasets_5p0.subject_id_getter" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets/feat_level3_sst_hp_alldatasets_5p0/subject_id_getter".
230206-13:26:47,13 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp_alldatasets_5p0.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets/feat_level3_sst_hp_alldatasets_5p0/_contrast_n_3_fwhm_5p0_model_n_0/selector".
230206-13:26:47,13 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp_alldatasets_5

230206-13:26:48,962 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp_alldatasets_5p0.copemerge" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets/feat_level3_sst_hp_alldatasets_5p0/_contrast_n_2_fwhm_5p0_model_n_0/copemerge".
230206-13:26:48,963 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp_alldatasets_5p0.copemerge" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets/feat_level3_sst_hp_alldatasets_5p0/_contrast_n_3_fwhm_5p0_model_n_0/copemerge".
230206-13:26:48,965 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp_alldatasets_5p0.copemerge" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets/feat_level3_sst_hp_alldatasets_5p0/_contrast_n_5_fwhm_5p0_model_n_0/copemerge".
230206-13:26:48,965 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp_alldatasets_5p0.copemerge" in "/home/scotti/projects/3t_7t_sst_com

230206-13:26:50,967 nipype.workflow INFO:
	 [Node] Finished "l3model", elapsed time 0.002021s.
230206-13:26:50,968 nipype.workflow INFO:
	 [Node] Executing "l3model" <nipype.interfaces.fsl.model.MultipleRegressDesign>
230206-13:26:50,970 nipype.workflow INFO:
	 [Node] Executing "l3model" <nipype.interfaces.fsl.model.MultipleRegressDesign>
230206-13:26:50,970 nipype.workflow INFO:
	 [Node] Executing "l3model" <nipype.interfaces.fsl.model.MultipleRegressDesign>
230206-13:26:50,971 nipype.workflow INFO:
	 [Node] Executing "l3model" <nipype.interfaces.fsl.model.MultipleRegressDesign>
230206-13:26:50,972 nipype.workflow INFO:
	 [Node] Finished "l3model", elapsed time 0.002734s.
230206-13:26:50,971 nipype.workflow INFO:
	 [Node] Executing "l3model" <nipype.interfaces.fsl.model.MultipleRegressDesign>
230206-13:26:50,976 nipype.workflow INFO:
	 [Node] Finished "l3model", elapsed time 0.00528s.
230206-13:26:50,976 nipype.workflow INFO:
	 [Node] Finished "l3model", elapsed time 0.005068s.
230206

230206-13:29:57,140 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp_alldatasets_5p0.flameo" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets/feat_level3_sst_hp_alldatasets_5p0/_contrast_n_5_fwhm_5p0_model_n_0/_run_mode_flame12/flameo".
230206-13:29:57,141 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp_alldatasets_5p0.flameo" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets/feat_level3_sst_hp_alldatasets_5p0/_contrast_n_5_fwhm_5p0_model_n_0/_run_mode_flame1/flameo".
230206-13:29:57,141 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp_alldatasets_5p0.flameo" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets/feat_level3_sst_hp_alldatasets_5p0/_contrast_n_3_fwhm_5p0_model_n_0/_run_mode_flame12/flameo".
230206-13:29:57,143 nipype.workflow INFO:
	 [Node] Executing "flameo" <nipype.interfaces.fsl.model.FLAMEO>
230206

230206-14:19:10,521 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate1", elapsed time 0.493049s.
230206-14:19:10,525 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate5", elapsed time 0.493262s.
230206-14:19:10,527 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate6", elapsed time 0.492363s.
230206-14:19:10,531 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate4", elapsed time 0.500775s.
230206-14:19:10,534 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate3", elapsed time 0.504176s.
230206-14:19:10,536 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate2", elapsed time 0.506449s.
230206-14:19:10,547 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate0", elapsed time 0.519327s.
230206-14:19:11,985 nipype.workflow INFO:
	 [Job 104] Completed (_smoothestimate0).
230206-14:19:11,986 nipype.workflow INFO:
	 [Job 105] Completed (_smoothestimate1).
230206-14:19:11,986 nipype.workflow INFO:
	 [Job 106] Completed (_smoothestimate2).
230206-14:19:11,987 

...

230208-12:37:28,929 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster4" - collecting precomputed outputs
230208-12:37:28,929 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster4" - collecting precomputed outputs
230208-12:37:28,929 nipype.workflow INFO:
	 [Node] "_grf_cluster4" found cached.
230208-12:37:28,929 nipype.workflow INFO:
	 [Node] "_grf_cluster4" found cached.
230208-12:37:28,930 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster5" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets/feat_level3_sst_hp_alldatasets_5p0/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_3.1/grf_cluster/mapflow/_grf_cluster5".
230208-12:37:28,931 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster5" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets/feat_level3_sst_hp_alldatasets_5p0/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_2.3/grf_cluster/mapflow/_grf_cluster5".
2302

230208-12:37:30,960 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/third_level_model/level3_zstats/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/zstat7.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-7_flame-12_desc-zstat.nii.gz
230208-12:37:30,960 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/third_level_model/level3_zstats/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/zstat5.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-5_flame-12_desc-zstat.nii.gz
230208-12:37:30,960 nipype.inter

230208-12:37:30,966 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/third_level_model/level3_copes/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/cope5.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-5_flame-12_desc-cope.nii.gz
230208-12:37:30,966 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/third_level_model/level3_varcopes/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/varcope3.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-3_flame-12_desc-varcope.nii.gz
230208-12:37:30,967 nipype.in

230208-12:37:30,972 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/third_level_model/level3_tdof_ts/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/tdof_t5.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-5_flame-12_desc-tdof_t.nii.gz
230208-12:37:30,973 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/third_level_model/level3_varcopes/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/varcope6.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-6_flame-12_desc-varcope.nii.gz
230208-12:37:30,973 nip

230208-12:37:30,978 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/third_level_model/level3_tdof_ts/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/tdof_t7.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-7_flame-12_desc-tdof_t.nii.gz
230208-12:37:30,978 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/third_level_model/grf_thresholded_zstats_file/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_2.3/_grf_cluster6/zstat7_threshold.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/all_datasets/fwhm-5p0/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-7_fla

## DUMMY variables code

In [13]:
import pandas as pd

url = "https://stats.idre.ucla.edu/stat/data/hsb2.csv"
hsb2 = pd.read_table(url, delimiter=",")

In [14]:
hsb2.head(10)


,id,female,race,ses,schtyp,prog,read,write,math,science,socst
0,70,0,4,1,1,1,57,52,41,47,57
1,121,1,4,2,1,3,68,59,53,63,61
2,86,0,4,3,1,1,44,33,54,58,31
3,141,0,4,3,1,3,63,44,47,53,56
4,172,0,4,2,1,2,47,52,57,53,61
5,113,0,4,2,1,2,44,52,51,63,61
6,50,0,3,2,1,1,50,59,42,53,61
7,11,0,1,2,1,2,34,46,45,39,36
8,84,0,4,2,1,1,63,57,54,58,51
9,48,0,3,2,1,2,57,55,52,50,51


In [15]:
hsb2.groupby("race")["write"].mean()

race
1    46.458333
2    58.000000
3    48.200000
4    54.055172
Name: write, dtype: float64

In [16]:
from patsy.contrasts import Treatment

levels = [1, 2, 3, 4]
contrast = Treatment(reference=0).code_without_intercept(levels)
print(contrast.matrix)

[[0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [17]:
hsb2.race.head(10)


0    4
1    4
2    4
3    4
4    4
5    4
6    3
7    1
8    4
9    3
Name: race, dtype: int64

In [18]:
print(contrast.matrix[hsb2.race - 1, :][:20])


[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [19]:
pd.get_dummies(hsb2.race.values, drop_first=False)


,1,2,3,4
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
195,0,1,0,0
196,0,0,0,1
197,0,0,0,1
198,0,0,0,1


In [20]:
hsb2

,id,female,race,ses,schtyp,prog,read,write,math,science,socst
0,70,0,4,1,1,1,57,52,41,47,57
1,121,1,4,2,1,3,68,59,53,63,61
2,86,0,4,3,1,1,44,33,54,58,31
3,141,0,4,3,1,3,63,44,47,53,56
4,172,0,4,2,1,2,47,52,57,53,61
...,...,...,...,...,...,...,...,...,...,...,...
195,31,1,2,2,2,1,55,59,52,42,56
196,145,1,4,2,1,3,42,46,38,36,46
197,187,1,4,2,2,1,57,41,57,55,52
198,118,1,4,2,1,1,55,62,58,58,61


In [22]:
Treatment

patsy.contrasts.Treatment

In [21]:
from statsmodels.formula.api import ols

mod = ols("write ~ C(race, Treatment)", data=hsb2)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  write   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     7.833
Date:                Mon, 03 Oct 2022   Prob (F-statistic):           5.78e-05
Time:                        16:21:08   Log-Likelihood:                -721.77
No. Observations:                 200   AIC:                             1452.
Df Residuals:                     196   BIC:                             1465.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

simple coding

In [23]:
from patsy.contrasts import ContrastMatrix


def _name_levels(prefix, levels):
    return ["[%s%s]" % (prefix, level) for level in levels]


class Simple(object):
    def _simple_contrast(self, levels):
        nlevels = len(levels)
        contr = -1.0 / nlevels * np.ones((nlevels, nlevels - 1))
        contr[1:][np.diag_indices(nlevels - 1)] = (nlevels - 1.0) / nlevels
        return contr

    def code_with_intercept(self, levels):
        contrast = np.column_stack(
            (np.ones(len(levels)), self._simple_contrast(levels))
        )
        return ContrastMatrix(contrast, _name_levels("Simp.", levels))

    def code_without_intercept(self, levels):
        contrast = self._simple_contrast(levels)
        return ContrastMatrix(contrast, _name_levels("Simp.", levels[:-1]))

In [24]:
hsb2.groupby("race")["write"].mean().mean()


51.67837643678162

In [25]:
contrast = Simple().code_without_intercept(levels)
print(contrast.matrix)

[[-0.25 -0.25 -0.25]
 [ 0.75 -0.25 -0.25]
 [-0.25  0.75 -0.25]
 [-0.25 -0.25  0.75]]


In [33]:
mod = ols("write ~ C(race, contrast)", data=hsb2)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  write   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     7.833
Date:                Mon, 03 Oct 2022   Prob (F-statistic):           5.78e-05
Time:                        16:27:40   Log-Likelihood:                -721.77
No. Observations:                 200   AIC:                             1452.
Df Residuals:                     196   BIC:                             1465.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [29]:
contrast

ContrastMatrix(array([[-0.25, -0.25, -0.25],
                      [ 0.75, -0.25, -0.25],
                      [-0.25,  0.75, -0.25],
                      [-0.25, -0.25,  0.75]]),
               ['[Simp.1]', '[Simp.2]', '[Simp.3]'])

In [30]:
ols

<bound method Model.from_formula of <class 'statsmodels.regression.linear_model.OLS'>>

In [34]:
C(race)

NameError: name 'C' is not defined

In [14]:
def get_design_matrix(second_level_contrast, dataset_ids, subject_ids): #=subject_ids):
    print(second_level_contrast)
    
    import numpy as np
    from patsy.contrasts import ContrastMatrix
    from collections import Counter
    
    def _name_levels(prefix, levels):
        return ["[%s%s]" % (prefix, level) for level in levels]

    class Simple(object):
        def _simple_contrast(self, levels):
            nlevels = len(levels)
            contr = -1.0 / nlevels * np.ones((nlevels, nlevels - 1))
            contr[1:][np.diag_indices(nlevels - 1)] = (nlevels - 1.0) / nlevels
            return contr

        def code_with_intercept(self, levels):
            contrast = np.column_stack(
                (np.ones(len(levels)), self._simple_contrast(levels))
            )
            return ContrastMatrix(contrast, _name_levels("Simp.", levels))

        def code_without_intercept(self, levels):
            contrast = self._simple_contrast(levels)
            return ContrastMatrix(contrast, _name_levels("Simp.", levels[:-1]))
        
    levels = [1,2,3,4,5]
    contrast = Simple().code_without_intercept(levels)
    
    regressors = {'intercept': [1 for x in range(len(subject_ids))],
                  'SM': list(np.repeat([item[0] for item in contrast.matrix],list(Counter(dataset_ids).values()))),
                  'SJSI': list(np.repeat([item[0] for item in contrast.matrix],list(Counter(dataset_ids).values()))),
                  'aron': list(np.repeat([item[0] for item in contrast.matrix],list(Counter(dataset_ids).values()))),
                  'openfmri': list(np.repeat([item[0] for item in contrast.matrix],list(Counter(dataset_ids).values())))}
    
    # contrasts (3rd-level)
    third_level_contrasts = [('Group mean', 'T', ['intercept'], [1.0]),
                             ('-Group mean', 'T', ['intercept'], [-1.0])
                             ]
    
    return third_level_contrasts, regressors

In [11]:
# from patsy.contrasts import ContrastMatrix
# import numpy as np

def _name_levels(prefix, levels):
    return ["[%s%s]" % (prefix, level) for level in levels]


class Simple(object):
    def _simple_contrast(self, levels):
        nlevels = len(levels)
        contr = -1.0 / nlevels * np.ones((nlevels, nlevels - 1))
        contr[1:][np.diag_indices(nlevels - 1)] = (nlevels - 1.0) / nlevels
        return contr

    def code_with_intercept(self, levels):
        contrast = np.column_stack(
            (np.ones(len(levels)), self._simple_contrast(levels))
        )
        return ContrastMatrix(contrast, _name_levels("Simp.", levels))

    def code_without_intercept(self, levels):
        contrast = self._simple_contrast(levels)
        return ContrastMatrix(contrast, _name_levels("Simp.", levels[:-1]))

In [17]:
levels = [1,2,3,4,5]
contrast = Simple().code_without_intercept(levels)
print(contrast.matrix)

[[-0.2 -0.2 -0.2 -0.2]
 [ 0.8 -0.2 -0.2 -0.2]
 [-0.2  0.8 -0.2 -0.2]
 [-0.2 -0.2  0.8 -0.2]
 [-0.2 -0.2 -0.2  0.8]]


In [2]:
dummies = [[-1,-1,-1,-1],[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]

In [ ]:
third, reg = get_design_matrix(1, dataset_ids, subject_ids)

In [17]:
levels = [1,2,3,4,5]

In [23]:
np.repeat(levels, list(Counter(s).values()))

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [15]:
d, s = get_subject_ids()

In [20]:
list(Counter(dataset_ids).values())

[1, 1, 1, 1, 1]

In [22]:
d

('BI3T',
 'FMFT',
 'GAIT',
 'KCAT',
 'KP6T',
 'LV2T',
 'MRCT',
 'NM3T',
 'PF5T',
 'RSIT',
 'SC1T',
 'SPGT',
 'TS6T',
 'UM2T',
 'VL1T',
 'WW2T',
 'ZK4T',
 '01',
 '02',
 '03',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '13',
 '15',
 '16',
 '17',
 '18',
 '002',
 '003',
 '005',
 '006',
 '007',
 '009',
 '011',
 '012',
 '014',
 '015',
 '016',
 '017',
 '018',
 '020',
 '021',
 '023',
 '024',
 '025',
 '026',
 '029',
 '031',
 '032',
 '033',
 '034',
 '035',
 '037',
 '038',
 '039',
 '040',
 '041',
 '042',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '10159',
 '10171',
 '10206',
 '10217',
 '10228',
 '10235',
 '10249',
 '10273',
 '10274',
 '10280',
 '10290',
 '10292',
 '10304',
 '10316',
 '10321',
 '10325',
 '10329',
 '10339',
 '10340',
 '10345',
 '10347',
 '10356',
 '10361',
 '10365',
 '10388',
 '10429',
 '10438',
 '10440',
 '10448',
 '10455',
 '10460',
 '10471',
 '10478',
 '10487',
 '10492',
 '10506',
 '10517',
 '10523',
 '10524',
 '1052

In [19]:
list(np.repeat([item[1] for item in contrast.matrix],list(Counter(d).values())))

[-0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -

In [20]:
list(np.repeat([item[3] for item in contrast.matrix],list(Counter(d).values())))

[-0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 -0.2,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,

In [5]:
dummies[0][0]

-1